### IMPORTING LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### IMPORTING DATASETS

In [2]:
links=pd.read_csv("D:DataSets/movies/links.csv")
movies=pd.read_csv("D:DataSets/movies/movies.csv")
ratings=pd.read_csv("D:DataSets/movies/ratings.csv")
tags=pd.read_csv("D:DataSets/movies/tags.csv")

#### DATA MERGING

In [3]:
movie=[links,movies,ratings,tags]
for i in range(0,len(movie)-2):
    for j in movie[i].columns:
        if j in movie[i+1].columns:
            df=movie[i].merge(movie[i+1],on=j,how='outer')
            df = df[df.isnull().sum(axis=1) ==0]

In [4]:
df.drop('timestamp',axis=1,inplace=True)

In [5]:
df.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100853
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100836 non-null  int64  
 1   title    100836 non-null  object 
 2   genres   100836 non-null  object 
 3   userId   100836 non-null  float64
 4   rating   100836 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 4.6+ MB


### ON THE BASIS OF RATING 

In [7]:
df_rating=df.groupby(by = ['title'])['rating'].count().reset_index().rename(columns = {'rating': 'totalRatingCount'})

In [8]:
df_rating.sort_values('totalRatingCount', ascending=False).head(10)

,title,totalRatingCount
3158,Forrest Gump (1994),329
7593,"Shawshank Redemption, The (1994)",317
6865,Pulp Fiction (1994),307
7680,"Silence of the Lambs, The (1991)",279
5512,"Matrix, The (1999)",278
8001,Star Wars: Episode IV - A New Hope (1977),251
4662,Jurassic Park (1993),238
1337,Braveheart (1995),237
8363,Terminator 2: Judgment Day (1991),224
7421,Schindler's List (1993),220


### USER BASED COLLABORATIVE FILTERING 

In [9]:
movie_features_df=df.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()

userId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,601.0,602.0,603.0,604.0,605.0,606.0,607.0,608.0,609.0,610.0
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### USING KNN ALGORITHM 

In [10]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [11]:
movie=input("Enter the movie you want recommendations for ")

Enter the movie you want recommendations for xXx: State of the Union (2005)


In [12]:
for index,name in enumerate(movie_features_df.index):
    if name==movie:
        test=index
print(test)
distance, indices = model_knn.kneighbors(movie_features_df.iloc[test,:].values.reshape(1, -1), n_neighbors = 6)

for i in range(0, len(distance.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[test]))
    else:
        print('{0}: {1}'.format(i, movie_features_df.index[indices.flatten()[i]]))

9716
Recommendations for xXx: State of the Union (2005):

1: Taxi (2004)
2: Pulse (2006)
3: Lakeview Terrace (2008)
4: Darkness Falls (2003)
5: Hills Have Eyes II, The (2007)
